In [27]:
from codice.mymodel import modello_albero
import pyomo.environ as pyo
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 
from sklearn.model_selection import train_test_split
from codice.mytree import Tree

SEED = 1

### PREPROCESS DATASET ###

# TODO: NORMALIZZARE DATI
raw_data = loadarff('../dataset/Rice_Cammeo_Osmancik.arff')
df_data = pd.DataFrame(raw_data[0]).to_numpy()

# TODO FIXARE ! SONO TUTTI UNI
x_data_raw = df_data[:, :-1]
# Cammeo = 0, Osmancik = 1
y_data_raw = [0 if x.decode("utf-8") == "Cammeo" else 1 for x in df_data[:,-1]]

X_train, X_test, y_train, y_test = train_test_split(x_data_raw, y_data_raw, test_size=0.25, stratify=y_data_raw, 
                                                        random_state=SEED) 

num_features = len(X_train[0])


In [29]:
### BUILD PARAMETERS ###
alpha = 0 # complexity
beta = 0 # n min of elements in a leaf
delta = 0 # depth

# compute epsilon value (uj)
u = [None for _ in range(num_features)]
for j in range(num_features):
    for i in range(len(X_train)-1):
        diff = abs(X_train[i][j] - X_train[i+1][j])
        if u[j] == None or diff < u[j]:
            u[j] = diff
umax = max(u)
umin = min(u)

## TODO: capire come calcolarlo
lcap = 0.5 

yi_class = [[None for _ in range(len(np.unique(y_train)))] for _ in range(len(X_train))]
for i in range(len(X_train)):
    for col in range(len(np.unique(y_train))):
        yi_class[i][col] = 1 if y_train[i] == col else 0


### BUILD TREE ###
tree = Tree(delta)

In [ ]:


modello = modello_albero(alpha, beta, delta, u, umax, umin, X_train, y_train, tree, lcap, yi_class)

opt = pyo.SolverFactory('gurobi')
results = opt.solve(modello) # solves and updates model
pyo.assert_optimal_termination(results)

In [ ]:
### VALIDAZIONE ###

